In [1]:
# set to data directory that contains the images
datadir = '/Users/Will Hecmanczuk/OneDrive/Desktop/SBEVE/bass/'

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision.models as models
from PIL import Image


In [4]:
# This function converts the (x lbs, x oz) format to x lbs
def convert_to_pounds(pounds, ounces):
    pounds = float(pounds)
    ounces = float(ounces)
    weight_in_pounds = pounds + (ounces / 16.0)
    return weight_in_pounds


In [19]:
import os
# This function renames bass image files to x.png where x is an integer from 0-99
def rename_files(directory):
    files = os.listdir(directory)
    for i, file_name in enumerate(sorted(files)):
        file_ext = os.path.splitext(file_name)[1]
        new_name = f"{i}{file_ext}"
        old_path = os.path.join(directory, file_name)
        new_path = os.path.join(directory, new_name)
        os.rename(old_path, new_path)
        print(f"Renamed {file_name} to {new_name}")

# Specify the directory containing the files
image_directory = datadir + "bass_images/"


rename_files(image_directory)



Renamed IMG_3096.PNG to 0.PNG
Renamed IMG_3098.PNG to 1.PNG
Renamed IMG_3100.PNG to 2.PNG
Renamed IMG_3101.PNG to 3.PNG
Renamed IMG_3102.PNG to 4.PNG
Renamed IMG_3103.PNG to 5.PNG
Renamed IMG_3104.PNG to 6.PNG
Renamed IMG_3105.PNG to 7.PNG
Renamed IMG_3106.PNG to 8.PNG
Renamed IMG_3107.PNG to 9.PNG
Renamed IMG_3108.PNG to 10.PNG
Renamed IMG_3109.PNG to 11.PNG
Renamed IMG_3110.PNG to 12.PNG
Renamed IMG_3111.PNG to 13.PNG
Renamed IMG_3112.PNG to 14.PNG
Renamed IMG_3113.PNG to 15.PNG
Renamed IMG_3115.PNG to 16.PNG
Renamed IMG_3116.PNG to 17.PNG
Renamed IMG_3117.PNG to 18.PNG
Renamed IMG_3118.PNG to 19.PNG
Renamed IMG_3119.PNG to 20.PNG
Renamed IMG_3120.PNG to 21.PNG
Renamed IMG_3121.PNG to 22.PNG
Renamed IMG_3122.PNG to 23.PNG
Renamed IMG_3123.PNG to 24.PNG
Renamed IMG_3124.PNG to 25.PNG
Renamed IMG_3125.PNG to 26.PNG
Renamed IMG_3126.PNG to 27.PNG
Renamed IMG_3127.PNG to 28.PNG
Renamed IMG_3128.PNG to 29.PNG
Renamed IMG_3129.PNG to 30.PNG
Renamed IMG_3130.PNG to 31.PNG
Renamed IMG_3132.P

# Below Is the first attempt

# take 2

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models
from PIL import Image
import os
from torchvision import transforms

# Define the model architecture
model = models.resnet18(pretrained=True)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 1)

# Define the dataset class
class BassDataset(Dataset):
    def __init__(self, image_folder, weights_file):
        self.image_folder = image_folder
        self.weights = self.load_weights(weights_file)

    def load_weights(self, weights_file):
        weights = []
        with open(weights_file, 'r') as file:
            for line in file:
                line = line.strip()
                if line:
                    pounds, ounces = map(int, line.split(','))
                    weight = convert_to_pounds(pounds, ounces)
                    weights.append(weight)
        return weights

    def __len__(self):
        return len(self.weights)

    def __getitem__(self, index):
        image_path = os.path.join(self.image_folder, f"{index}.png")
        image = Image.open(image_path).convert("RGB")
        weight = self.weights[index]

        return image, weight

# Define paths to the image folder and weights file
image_folder = datadir + "bass_images/"
weights_file = datadir + "bass_weights.txt"

# Create dataset instance
dataset = BassDataset(image_folder, weights_file)

# Split the dataset into training(80) and validation(20) sets
train_size = int(0.8 * len(dataset))
valid_size = len(dataset) - train_size
train_dataset, valid_dataset = torch.utils.data.random_split(dataset, [train_size, valid_size])

# Create data loaders
batch_size = 16

# Define the custom collate function
def collate_fn(batch):
    images, weights = zip(*batch)
    transform = transforms.ToTensor()
    images = [transform(image) for image in images]
    images = torch.stack(images)
    weights = torch.tensor(weights, dtype=torch.float32)
    return images, weights

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, collate_fn=collate_fn)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0

    for images, weights in train_loader:
        images = images.to(device)
        weights = weights.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs.squeeze(), weights)

        loss.backward()
        optimizer.step()

        train_loss += loss.item() * images.size(0)

    train_loss = train_loss / len(train_dataset)

    # Validation
    model.eval()
    valid_loss = 0.0

    with torch.no_grad():
        for images, weights in valid_loader:
            images = images.to(device)
            weights = weights.to(device)

            outputs = model(images)
            loss = criterion(outputs.squeeze(), weights)

            valid_loss += loss.item() * images.size(0)

    valid_loss = valid_loss / len(valid_dataset)

    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Valid Loss: {valid_loss:.4f}")



Epoch [1/10], Train Loss: 2.2882, Valid Loss: 154.3598
Epoch [2/10], Train Loss: 0.8839, Valid Loss: 37.5500
Epoch [3/10], Train Loss: 0.6748, Valid Loss: 11.2727
Epoch [4/10], Train Loss: 0.5774, Valid Loss: 4.6583
Epoch [5/10], Train Loss: 0.5605, Valid Loss: 1.9849
Epoch [6/10], Train Loss: 0.5117, Valid Loss: 1.6858
Epoch [7/10], Train Loss: 0.4558, Valid Loss: 4.1749
Epoch [8/10], Train Loss: 0.4300, Valid Loss: 2.2420
Epoch [9/10], Train Loss: 0.3564, Valid Loss: 1.8533
Epoch [10/10], Train Loss: 0.2714, Valid Loss: 1.9096


In [13]:
test = datadir + "IMG_3216.png"


In [14]:
# This function uses the model to predict the weight of a given bass image.
def predict_weight(image_path):
    transform = transforms.Compose([
        transforms.ToTensor(),])

    # Load and preprocess the image
    image = Image.open(image_path).convert("RGB")
    input_image = transform(image)

    # Add an extra dimension for batch
    input_image = input_image.unsqueeze(0)

    # Move the input image to the device (CPU or GPU)
    input_image = input_image.to(device)

    # Set the model to evaluation mode
    model.eval()

    # Perform the inference
    with torch.no_grad():
        output = model(input_image)

    # Get the predicted weight
    predicted_weight = output.item()

    return predicted_weight

In [15]:
predict_weight(test)

2.441661834716797